# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [78]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.72,96,100,9.14,SH,1697284059
1,1,stanley,54.8680,-1.6985,10.71,58,0,9.26,GB,1697284053
2,2,waitangi,-43.9535,-176.5597,12.68,92,40,3.58,NZ,1697284062
3,3,sao joao da barra,-21.6403,-41.0511,22.05,87,100,6.02,BR,1697284198
4,4,albany,42.6001,-73.9662,8.23,91,100,1.20,US,1697283795


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = .75
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values

#between 26 and 18 celsius (65-80 fahrenheit)
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 26) & (city_data_df["Max Temp"] >= 18)]

# low humidity, I already live in virginia, I want to escape the swampy air
city_data_df = city_data_df.loc[(city_data_df["Humidity"] < 40)]

#not too windy
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,corinto,37.9407,22.9573,22.72,34,83,1.45,GR,1697284202
55,55,mount isa,-20.7333,139.5000,23.87,17,47,3.60,AU,1697284065
72,72,hotan,37.0997,79.9269,19.35,19,96,2.31,CN,1697284065
122,122,turpan,42.9333,89.1667,24.25,22,27,0.62,CN,1697284207
191,191,kuqa,41.7278,82.9364,20.46,27,94,2.06,CN,1697284213
206,206,alice springs,-23.7000,133.8833,18.75,13,0,3.09,AU,1697283782
215,215,santa cruz,-17.8000,-63.1667,23.03,38,75,4.12,BO,1697283929
272,272,qala i naw,34.9874,63.1289,18.47,30,2,3.33,AF,1697284221
446,446,broken hill,-31.9500,141.4333,20.09,37,0,2.92,AU,1697284098
545,545,matamba,-8.9833,33.9667,25.34,29,78,1.79,TZ,1697284245


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat= city_data_df["Lat"]
lng= city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{
        "City":city, 
        "Country":country, 
        "Lat": lat,
        "Lng": lng, 
        "Humidity": humidity
}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\Alex\AppData\Local\Temp\ipykernel_1792\3031869703.py:8: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{
C:\Users\Alex\AppData\Local\Temp\ipykernel_1792\3031869703.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
54,corinto,GR,37.9407,22.9573,34,
55,mount isa,AU,-20.7333,139.5000,17,
72,hotan,CN,37.0997,79.9269,19,
122,turpan,CN,42.9333,89.1667,22,
191,kuqa,CN,41.7278,82.9364,27,
206,alice springs,AU,-23.7000,133.8833,13,
215,santa cruz,BO,-17.8000,-63.1667,38,
272,qala i naw,AF,34.9874,63.1289,30,
446,broken hill,AU,-31.9500,141.4333,37,
545,matamba,TZ,-8.9833,33.9667,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [84]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"]= f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
corinto - nearest hotel: Apollon
mount isa - nearest hotel: Ibis Styles
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
turpan - nearest hotel: Oriental
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
alice springs - nearest hotel: Aurora Alice Springs
santa cruz - nearest hotel: Aviador Hotel Boutique
qala i naw - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
matamba - nearest hotel: Mama Izengo's Hotel
lent'ekhi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
54,corinto,GR,37.9407,22.9573,34,Apollon
55,mount isa,AU,-20.7333,139.5000,17,Ibis Styles
72,hotan,CN,37.0997,79.9269,19,Hua Yi International Hotel (accepts foreigners)
122,turpan,CN,42.9333,89.1667,22,Oriental
191,kuqa,CN,41.7278,82.9364,27,Kuqa Hotel - foreigners accepted
206,alice springs,AU,-23.7000,133.8833,13,Aurora Alice Springs
215,santa cruz,BO,-17.8000,-63.1667,38,Aviador Hotel Boutique
272,qala i naw,AF,34.9874,63.1289,30,No hotel found
446,broken hill,AU,-31.9500,141.4333,37,Ibis Styles
545,matamba,TZ,-8.9833,33.9667,29,Mama Izengo's Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [87]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = .75,
    hover_cols =["Hotel Name", "Country"]
)


# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)